In [ ]:
#| default_exp model.roi

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
import cv2
import timm
import torch
import pickle
import random
import torch.nn as nn
import torch.nn.functional as F
from license_plate_detection.utils import *
from fastai.vision.all import *
from torchvision.ops import roi_align
from license_plate_detection.fpn import *

In [ ]:
#| export
class ROI(nn.Module):
    def __init__(self, in_channels, num_classes, roi_feat_size=(7,7)):
        super().__init__()
        self.roi_head= roi_align
        self.roi_feat_size= roi_feat_size
        self.relu= nn.ReLU()
        self.shared_conv= nn.Sequential(*[ConvLayer(in_channels,256,3,2,1), 
                                          Flatten()])
        self.reg_conv= nn.Sequential(*[nn.Linear(4096,num_classes*4)])
        self.cls_conv= nn.Sequential(*[nn.Linear(4096,num_classes+1)])
        
    def get_roi_gt(self, rois, gt_bboxes, gt_labels, size):
        rois= TensorPoint(rois[:,:,1:].detach())
        gt_bboxes= unscale_pnts(gt_bboxes, size)
        gt_roi_cls, gt_roi_reg=[],[]
        for idx,_ in enumerate(rois):
            flags= assign_anchor_torch(rois[idx], 
                                    gt_bboxes[:idx+1], 
                                    pos_iou_thr=0.8, 
                                    neg_iou_thr=0.2)
            roi_cls= sample_anchor_torch(rois[idx], flags, 5)
            roi_cls[gt_roi_cls==1]= gt_labels[idx]
            roi_reg= calc_offset_torch(gt_bboxes[idx], rois[idx])
            gt_roi_cls.append(roi_cls)
            gt_roi_reg.append(roi_reg)
        return torch.stack(gt_roi_cls), torch.stack(gt_roi_reg)
    
    def forward(self, x, rois):
        rois= rois[:,:, [0,2,1,4,3]].contiguous()
        roi_reg_feats, roi_cls_feats=[],[]
        for r in rois:
            roi_feats= self.roi_head(x, r, self.roi_feat_size, spatial_scale=0.125)
            shared_feats= self.shared_conv(roi_feats)
            reg_feats= self.reg_conv(shared_feats)
            cls_feats= self.cls_conv(shared_feats)
            cls_feats= F.softmax(cls_feats, dim=1)
            roi_reg_feats.append(reg_feats)
            roi_cls_feats.append(cls_feats)
        return torch.stack(roi_reg_feats), torch.stack(roi_cls_feats)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()